In [97]:
# from transformers import MarianMTModel, MarianTokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
# from transformers import pipeline 
import pandas as pd
from tqdm import tqdm

In [ ]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

In [27]:
article_en = "fuck you"

model_inputs = tokenizer(article_en, return_tensors="pt")

# translate from English to Kazakh
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["kk_KZ"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['иесірткі']

In [ ]:
def translate_to_kazakh(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    inputs = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["kk_KZ"]  # Kazakh Cyrillic
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]


In [117]:
df = pd.read_csv("../data/processed/combined_balanced.csv")

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40766 entries, 0 to 40765
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    40766 non-null  int64 
 1   clean_text    40766 non-null  object
 2   has_toxicity  40766 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 955.6+ KB


In [85]:
def translate_to_kazakh(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    try:
        inputs = tokenizer(text, return_tensors="pt")
        tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id["kk_KZ"]
        )
        return tokenizer.decode(tokens[0], skip_special_tokens=True)
    except Exception as e:
        print(f"Error on: {text[:30]}... → {e}")
        return ""

In [86]:
df

,Unnamed: 0,clean_text,has_toxicity
0,689,nerds attempt to block it? this is communist.,1
1,3829,""" style""""background-colorf5fffa padding0"""" cel...",0
2,2283,rt charlie sheen engaged to porn star mens hea...,0
3,3741,"taxonomic error mr. shyamal, who is written in...",0
4,14902,"now that i think about it, i led three lives i...",0
...,...,...,...
95,19158,website www.faisalabad.gov.pk,0
96,17902,"""this sentence is accurate then approach or ap...",0
97,4992,no wonder your career is in the dump that's wh...,1
98,22522,""" well, i really have only two concerns metros...",0


In [87]:
df["text_kk"] = df["clean_text"].apply(translate_to_kazakh)


KeyboardInterrupt: 

In [110]:
def count_chars(text):
    return len(text)

In [119]:
df.count()

Unnamed: 0      40766
clean_text      40766
has_toxicity    40766
dtype: int64

In [148]:
texts = df["clean_text"].sample(1688, random_state=42).fillna("").astype(str)
count_char = texts.apply(count_chars)
print(count_char.sum())

498159


In [93]:
# Parameters
batch_size = 8
translated_texts = []


for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
    batch = texts[i:i + batch_size]
    # 🔍 Clean up batch: remove empty strings
    batch = [text for text in batch if text.strip() != ""]
    if len(batch) == 0:
        translated_texts.extend([""] * batch_size)
        continue

    try:
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        outputs = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id["kk_KZ"]
        )
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    except Exception as e:
        print(f"⚠️ Error on batch {i}-{i+batch_size}: {e}")
        decoded = [""] * len(batch)

    translated_texts.extend(decoded)

# Fix length mismatch if needed
while len(translated_texts) < len(texts):
    translated_texts.append("")

df["text_kk"] = translated_texts

Translating: 100%|██████████| 3/3 [00:26<00:00,  8.84s/it]


In [94]:
df

,Unnamed: 0,clean_text,has_toxicity,text_kk
0,12448,lolz try harder plz? comedrinker,1,lolz бұдан да қатты ұмтылайық Plzeň? Comedrinker
1,856,i have evry right to be pissed off!!!! even wh...,1,Мен наразы болу үшін қате іс бар!!!! Тіпті мен...
2,26106,what kind of a stupid name is bongwarrior anyw...,1,"Bongwarrior ма сонда да ақымақ, сонда да ақымақ."
3,1013,""" for everyone's info i don't give a flying fu...",1,account-ты алып тастамаса бұл мейрамханаға өші...
4,22419,""" , delaware county generally posts county rou...",0,Делавэр округі әдетте жергілікті кепілдік марк...
5,5100,rt i'm on my straight grizzly so why would i b...,1,"Тіпті би оңшыл ергежейге сүйеніпkam, තොට лайық..."
6,20034,""" no, you need to fuck off. talk """,1,"-Жоқ, солай іштеңе жаса, сөйтіп көр."
7,28303,"aluminum come on, not even aluminum is safe fo...",0,"алюминий бола тұра, алюминийдің өзінде өте қуа..."
8,8035,spirituality thought you'd be interested in this.,0,"ғаниеттілік туралы ойласақ, мұны ықтимал деп о..."
9,3774,not protected in the state of utah. there is n...,1,utah штатында олардың қорғалмауы керек. Күер е...
